In [1]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.12 ; cuda:  cu113


In [2]:
#pip install  "torch>=1.4.0,  torchvision>=0.5.0"

In [3]:
# Cài đặt thư viện underthesea để thực hiện word segment
!pip install underthesea 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.1 MB/s 
     |████████████████████████████████| 581 kB 74.8 MB/s 
     |████████████████████████████████| 965 kB 56.2 MB/s 
     |████████████████████████████████| 235 kB 74.3 MB/s 


In [4]:
# Cài đặt pytorch
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Cài đặt hugging face
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 60.2 MB/s 
     |████████████████████████████████| 6.6 MB 42.5 MB/s 


# Sentiment Analysis - Tripadvisor

## Use PhoBERT pre-train model to fine-tune

In [6]:
import underthesea
from transformers import AutoModel, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
import re
import csv
import os
import json
import torch
import transformers

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [10]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [11]:
os.environ['CUDA_VISIBLE_DEVICES']='2,3'

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
# Chuẩn hoá câu
def standardize_data(row):
    # remove stopword
    # remove . ? , at index final
    row =  re.sub(r'[\.,\?]+$-','', row)
    # remoce all .,"... in sentences
    row = row.replace(","," ").replace("."," ")\
    .replace(";"," ").replace("."," ")\
    .replace(":"," ").replace("."," ")\
    .replace('"'," ").replace("''"," ")\
    .replace("!"," ").replace("?"," ")\
    .replace("-"," ").replace("&"," ").replace("#"," ")
    
    row = row.strip()
    return row

In [14]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [15]:
%cd '/content/gdrive/My Drive/MDS0_k278_VuThiHa/Cau_3/'

/content/gdrive/My Drive/MDS0_k278_VuThiHa/Cau_3


In [16]:
df = pd.read_csv('process_data.csv', index_col=0)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82380 entries, 0 to 82379
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            82378 non-null  object
 1   rating          82380 non-null  int64 
 2   sentiment       82380 non-null  object
 3   output          82380 non-null  int64 
 4   processed_text  81778 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.8+ MB


In [18]:
df.head(2)

,text,rating,sentiment,output,processed_text
0,Quá Tuyệt Vời Khi Ở Des Arts Sài Gòn. #HôtelDe...,5,Positive,1,tuyệt_vời trải_nghiệm tuyệt_vời ghé tươi thích...
1,"Đáng đồng tiền!. Dịch vụ cao cấp, phong cách c...",5,Positive,1,đồng_tiền chuyên_nghiệp hơi thích_hợp chống tr...


In [19]:
df = df.dropna()

In [20]:
df["target"] = [ 1 if x=='like' else 0 for x in df.sentiment]

In [21]:
df["title_content_new"] =  [standardize_data(x) for x in df.processed_text]

In [22]:
df.head(2)

,text,rating,sentiment,output,processed_text,target,title_content_new
0,Quá Tuyệt Vời Khi Ở Des Arts Sài Gòn. #HôtelDe...,5,Positive,1,tuyệt_vời trải_nghiệm tuyệt_vời ghé tươi thích...,0,tuyệt_vời trải_nghiệm tuyệt_vời ghé tươi thích...
1,"Đáng đồng tiền!. Dịch vụ cao cấp, phong cách c...",5,Positive,1,đồng_tiền chuyên_nghiệp hơi thích_hợp chống tr...,0,đồng_tiền chuyên_nghiệp hơi thích_hợp chống tr...


In [23]:
#...
#Load pretrain tokenizers
#...
tokenizer =  AutoTokenizer.from_pretrained("vinai/phobert-base")

#encode lines
tokenized = df.title_content_new.apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))

print('encode',tokenized[1])

#decode lines
print('decode',tokenizer.decode(tokenized[1]))

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


encode [0, 3119, 1564, 1329, 2732, 335, 1289, 2247, 987, 2]
decode <s> đồng_tiền chuyên_nghiệp hơi thích_hợp chống trải tiếc đọc </s>


In [24]:
# get length max of tokenized
max_len = 128 # max là 512

# if length of tokenized not equal max_len so padding value 0
padded = np.array([i + [0]*(max_len-len(i)) if len(i)<max_len else i[:max_len] for i in tokenized.values])
print("padded:", padded[1])
print("len padded:", padded.shape)

# np.savetxt('kaggle/working/padded.csv', padded, delimiter=",")

padded: [   0 3119 1564 1329 2732  335 1289 2247  987    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
len padded: (81778, 128)


In [25]:
# get attention mask (0: not has word, 1: has word)
attention_mask = np.where(padded ==0,0,1)
print("attention mask:", attention_mask[1])

# np.savetxt('kaggle/working/attention_mask.csv', attention_mask, delimiter=",")

attention mask: [0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [26]:
# Convert input to tensor
padded = torch.tensor(padded).to(device=device)
attention_mask = torch.tensor(attention_mask).to(device=device)

In [27]:
#Load pretrain model
model = AutoModel.from_pretrained("vinai/phobert-base").to(device)

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
# Train model
model.to (device = device)

with torch.no_grad():
    last_hidden_states = model(padded, attention_mask = attention_mask)

NameError: ignored

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
print('features:',features)

X_train, X_test, y_train, y_test = train_test_split(features, df["target"])

In [ ]:
type(features)
type(df["target"])

In [ ]:
np.savetxt("features.csv", features, delimiter=",")
# Reading the csv into an array
# features = np.genfromtxt("features.csv",delimiter=",")

In [ ]:
df['target'].to_csv("target.csv")

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [ ]:
cl = GaussianNB()
cl.fit(X_train, y_train)

# Save model
# joblib.dump(cl, 'save_model.pkl')
sc = cl.score(X_test, y_test)
print('score:', sc)

In [ ]:
lr = LogisticRegression(solve="lbfgs")
lr.fit(X_train, y_train)

# Save model
# joblib.dump(cl, 'save_model.pkl')
sc = lr.score(X_test, y_test)
print('score:', sc)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = lr.predict(X_test)
print('Classification Report')
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Save model
# joblib.dump(cl, 'save_model.pkl')
sc = rf.score(X_test, y_test)
print('score:', sc)

### Nhận xét:

In [ ]:
import pickle

In [ ]:
# save model
filename = 'trip_lr.model.pkl'
pickle.dump(lr, open(filename, 'wb'))
#load_model = pickle.load(open(filename, 'rb'))
